In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from termcolor import colored
import requests
import io
import time
import os
import json
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
from functools import lru_cache

class StockAnalyzer:
    def __init__(self):
        self.base_dir = 'stock_analysis'
        self.stock_lists_dir = os.path.join(self.base_dir, 'stock_lists')
        self.results_dir = os.path.join(self.base_dir, 'daily_results')
        self.delisted_cache_file = os.path.join(self.stock_lists_dir, 'delisted_stocks.json')
        self.setup_directories()
        self.load_delisted_stocks()
        self.session = requests.Session()

    # [Previous methods remain unchanged...]

    def calculate_ema(self, data, length=9, source='Close', offset=0):
        """
        Calculate Exponential Moving Average with specified parameters
        
        Args:
            data (pd.DataFrame): Historical price data
            length (int): Length of EMA period
            source (str): Source column for calculation
            offset (int): Offset for calculation
            
        Returns:
            pd.Series: EMA values
        """
        # First calculate SMA as the smoothing line
        sma = data[source].rolling(window=9).mean()  # Smoothing length of 9 as specified
        
        # Calculate the multiplier
        multiplier = 2 / (length + 1)
        
        # Initialize EMA with SMA
        ema = pd.Series(index=data.index, dtype=float)
        ema.iloc[:length-1] = np.nan
        ema.iloc[length-1] = sma.iloc[length-1]
        
        # Calculate EMA
        for i in range(length, len(data)):
            ema.iloc[i] = (data[source].iloc[i] - ema.iloc[i-1]) * multiplier + ema.iloc[i-1]
        
        # Apply offset if specified
        if offset != 0:
            ema = ema.shift(offset)
            
        return ema

    def get_stock_data(self, symbol, exchange='NSE', days=60):
        """Fetch detailed stock data with additional information"""
        if symbol in self.delisted_stocks[exchange]:
            return None, None, None, None

        try:
            ticker_symbol = f"{symbol}.NS" if exchange == 'NSE' else f"{symbol}.BO"
            ticker = yf.Ticker(ticker_symbol)
            
            # Get stock info
            info = ticker.info
            
            end_date = datetime.now()
            start_date = end_date - timedelta(days=days)

            hist_data = ticker.history(
                start=start_date,
                end=end_date,
                interval="1d"
            )

            if hist_data.empty or len(hist_data) < 20:
                self.delisted_stocks[exchange].append(symbol)
                self.save_delisted_stocks()
                return None, None, None, None

            # Calculate EMA
            hist_data['EMA_9'] = self.calculate_ema(hist_data, length=9, source='Close', offset=0)

            latest_price = hist_data['Close'].iloc[-1]
            prices_tuple = tuple(hist_data['Close'].values)
            latest_rsi = self.calculate_rsi(prices_tuple)

            return latest_price, hist_data, latest_rsi, info

        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")
            self.delisted_stocks[exchange].append(symbol)
            self.save_delisted_stocks()
            return None, None, None, None

    def process_stock_batch(self, batch_data):
        """Process a batch of stocks with enhanced information"""
        symbols, exchange = batch_data
        results = []
        
        with ThreadPoolExecutor(max_workers=10) as executor:
            futures = {executor.submit(self.get_stock_data, symbol, exchange): symbol 
                      for symbol in symbols}
            
            for future in futures:
                symbol = futures[future]
                try:
                    latest_price, hist_data, latest_rsi, info = future.result()
                    if all(v is not None for v in [latest_price, hist_data, latest_rsi, info]):
                        # Extract additional information safely using get() method
                        stock_info = {
                            'Symbol': symbol,
                            'Exchange': exchange,
                            'Company_Name': info.get('longName', 'N/A'),
                            'Latest_Price': latest_price,
                            'EMA_9': hist_data['EMA_9'].iloc[-1] if 'EMA_9' in hist_data else 'N/A',
                            'RSI': latest_rsi,
                            'Volume': hist_data['Volume'].iloc[-1] if 'Volume' in hist_data else 0,
                            'Market_Cap': info.get('marketCap', 'N/A'),
                            'PE_Ratio': info.get('trailingPE', 'N/A'),
                            'EPS': info.get('trailingEps', 'N/A'),
                            'Dividend_Yield': info.get('dividendYield', 'N/A'),
                            'Book_Value': info.get('bookValue', 'N/A'),
                            'Sector': info.get('sector', 'N/A'),
                            'Industry': info.get('industry', 'N/A'),
                            '52W_High': info.get('fiftyTwoWeekHigh', 'N/A'),
                            '52W_Low': info.get('fiftyTwoWeekLow', 'N/A'),
                            '50d_MA': info.get('fiftyDayAverage', 'N/A'),
                            '200d_MA': info.get('twoHundredDayAverage', 'N/A'),
                            'Beta': info.get('beta', 'N/A'),
                            'Previous_Close': info.get('previousClose', 'N/A'),
                            'Open': info.get('open', 'N/A'),
                            'Day_High': info.get('dayHigh', 'N/A'),
                            'Day_Low': info.get('dayLow', 'N/A'),
                            'Date': datetime.now().strftime('%Y-%m-%d')
                        }
                        
                        # Calculate additional metrics
                        if hist_data is not None:
                            stock_info.update({
                                'Volatility_30d': hist_data['Close'].pct_change().std() * np.sqrt(252),
                                'Returns_30d': (hist_data['Close'].iloc[-1] / hist_data['Close'].iloc[0] - 1) * 100,
                                'Average_Volume_30d': hist_data['Volume'].mean()
                            })
                        
                        results.append(stock_info)
                        
                except Exception as e:
                    print(f"Error processing {symbol}: {e}")

        return results

    # [Rest of the class methods remain unchanged...]

if __name__ == "__main__":
    analyzer = StockAnalyzer()
    analyzer.run_daily_analysis()

ema 9(above code)

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from termcolor import colored
import requests
import io
import time
import os
import json
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
from functools import lru_cache

class StockAnalyzer:

    def __init__(self):
            self.base_dir = 'stock_analysis'
            self.stock_lists_dir = os.path.join(self.base_dir, 'stock_lists')
            self.results_dir = os.path.join(self.base_dir, 'daily_results')
            self.delisted_cache_file = os.path.join(self.stock_lists_dir, 'delisted_stocks.json')
            self.setup_directories()
            self.load_delisted_stocks()
            self.session = requests.Session()  # Reuse session for better performance

    def setup_directories(self):
        """Create necessary directories if they don't exist"""
        os.makedirs(self.stock_lists_dir, exist_ok=True)
        os.makedirs(self.results_dir, exist_ok=True)
        
    def should_update_stock_list(self, exchange):
        """Check if stock list needs updating (weekly update)"""
        list_file = os.path.join(self.stock_lists_dir, f'{exchange}_stocks.json')
        if not os.path.exists(list_file):
            return True
        # Check if file is older than 7 days
        file_time = datetime.fromtimestamp(os.path.getmtime(list_file))
        return (datetime.now() - file_time).days >= 7
    
    def load_delisted_stocks(self):
        """Load previously identified delisted stocks"""
        try:
            with open(self.delisted_cache_file, 'r') as f:
                self.delisted_stocks = json.load(f)
        except FileNotFoundError:
            self.delisted_stocks = {'NSE': [], 'BSE': []}  # Changed from set to list
            self.save_delisted_stocks()

    def save_delisted_stocks(self):
        """Save delisted stocks to cache"""
        with open(self.delisted_cache_file, 'w') as f:
            json.dump({
                'NSE': self.delisted_stocks['NSE'],
                'BSE': self.delisted_stocks['BSE']
            }, f)
    
    def save_stock_list(self, symbols, exchange):
        """Save stock list to JSON file"""
        list_file = os.path.join(self.stock_lists_dir, f'{exchange}_stocks.json')
        data = {
            'symbols': symbols,
            'last_updated': datetime.now().strftime('%Y-%m-%d')
        }
        with open(list_file, 'w') as f:
            json.dump(data, f)

    def load_stock_list(self, exchange):
        """Load stock list from JSON file"""
        list_file = os.path.join(self.stock_lists_dir, f'{exchange}_stocks.json')
        with open(list_file, 'r') as f:
            data = json.load(f)
        return data['symbols']
    
    def download_nse_stocks(self):
        """Download NSE stock list if needed"""
        if self.should_update_stock_list('NSE'):
            try:
                url = "https://archives.nseindia.com/content/equities/EQUITY_L.csv"
                response = requests.get(url)
                df = pd.read_csv(io.StringIO(response.content.decode('utf-8')))
                symbols = df['SYMBOL'].tolist()
                self.save_stock_list(symbols, 'NSE')
                print("NSE stock list updated")
                return symbols
            except Exception as e:
                print(f"Error downloading NSE stocks: {e}")
                if os.path.exists(os.path.join(self.stock_lists_dir, 'NSE_stocks.json')):
                    return self.load_stock_list('NSE')
                return []
        return self.load_stock_list('NSE')

    def download_bse_stocks(self):
        """Download BSE stock list if needed"""
        if self.should_update_stock_list('BSE'):
            try:
                # Using the BSE list from static file or an appropriate source
                url = "https://www.bseindia.com/corporates/List_Scrips.aspx"
                response = requests.get(url)
                soup = BeautifulSoup(response.content, 'html.parser')
                table = soup.find('table', {'id': 'ContentPlaceHolder1_tblData'})
                symbols = []

                if table:
                    rows = table.find_all('tr')[1:]  # Skip header row
                    for row in rows:
                        cols = row.find_all('td')
                        if cols:
                            bse_code = cols[0].text.strip()
                            symbols.append(bse_code)

                if symbols:
                    self.save_stock_list(symbols, 'BSE')
                    print("BSE stock list updated")
                    return symbols
            except Exception as e:
                print(f"Error downloading BSE stocks: {e}")
                if os.path.exists(os.path.join(self.stock_lists_dir, 'BSE_stocks.json')):
                    return self.load_stock_list('BSE')
                return []
        return self.load_stock_list('BSE')

    @lru_cache(maxsize=1000)
    def calculate_rsi(self, prices_tuple, periods=14):
        """Calculate RSI with caching for performance"""
        prices = pd.Series(prices_tuple)
        delta = prices.diff()
        gain = delta.where(delta > 0, 0).rolling(window=periods).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=periods).mean()
        rs = gain / loss
        return float(100 - (100 / (1 + rs)).iloc[-1])

    def get_stock_data(self, symbol, exchange='NSE', days=60):
        """Fetch detailed stock data with additional information"""
        if symbol in self.delisted_stocks[exchange]:
            return None, None, None, None

        try:
            ticker_symbol = f"{symbol}.NS" if exchange == 'NSE' else f"{symbol}.BO"
            ticker = yf.Ticker(ticker_symbol)
            
            # Get stock info
            info = ticker.info
            
            end_date = datetime.now()
            start_date = end_date - timedelta(days=days)

            hist_data = ticker.history(
                start=start_date,
                end=end_date,
                interval="1d"
            )

            if hist_data.empty or len(hist_data) < 20:
                self.delisted_stocks[exchange].append(symbol)  # Changed from add to append
                self.save_delisted_stocks()  # Save immediately after adding
                return None, None, None, None

            latest_price = hist_data['Close'].iloc[-1]
            prices_tuple = tuple(hist_data['Close'].values)
            latest_rsi = self.calculate_rsi(prices_tuple)

            return latest_price, hist_data, latest_rsi, info

        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")
            self.delisted_stocks[exchange].append(symbol)  # Changed from add to append
            self.save_delisted_stocks()  # Save immediately after adding
            return None, None, None, None

    def process_stock_batch(self, batch_data):
        """Process a batch of stocks with enhanced information"""
        symbols, exchange = batch_data
        results = []
        
        with ThreadPoolExecutor(max_workers=10) as executor:
            futures = {executor.submit(self.get_stock_data, symbol, exchange): symbol 
                      for symbol in symbols}
            
            for future in futures:
                symbol = futures[future]
                try:
                    latest_price, hist_data, latest_rsi, info = future.result()
                    if all(v is not None for v in [latest_price, hist_data, latest_rsi, info]):
                        # Extract additional information safely using get() method
                        stock_info = {
                            'Symbol': symbol,
                            'Exchange': exchange,
                            'Company_Name': info.get('longName', 'N/A'),
                            'Latest_Price': latest_price,
                            'RSI': latest_rsi,
                            'Volume': hist_data['Volume'].iloc[-1] if 'Volume' in hist_data else 0,
                            'Market_Cap': info.get('marketCap', 'N/A'),
                            'PE_Ratio': info.get('trailingPE', 'N/A'),
                            'EPS': info.get('trailingEps', 'N/A'),
                            'Dividend_Yield': info.get('dividendYield', 'N/A'),
                            'Book_Value': info.get('bookValue', 'N/A'),
                            'Sector': info.get('sector', 'N/A'),
                            'Industry': info.get('industry', 'N/A'),
                            '52W_High': info.get('fiftyTwoWeekHigh', 'N/A'),
                            '52W_Low': info.get('fiftyTwoWeekLow', 'N/A'),
                            '50d_MA': info.get('fiftyDayAverage', 'N/A'),
                            '200d_MA': info.get('twoHundredDayAverage', 'N/A'),
                            'Beta': info.get('beta', 'N/A'),
                            'Previous_Close': info.get('previousClose', 'N/A'),
                            'Open': info.get('open', 'N/A'),
                            'Day_High': info.get('dayHigh', 'N/A'),
                            'Day_Low': info.get('dayLow', 'N/A'),
                            'Date': datetime.now().strftime('%Y-%m-%d')
                        }
                        
                        # Calculate additional metrics
                        if hist_data is not None:
                            stock_info.update({
                                'Volatility_30d': hist_data['Close'].pct_change().std() * np.sqrt(252),
                                'Returns_30d': (hist_data['Close'].iloc[-1] / hist_data['Close'].iloc[0] - 1) * 100,
                                'Average_Volume_30d': hist_data['Volume'].mean()
                            })
                        
                        results.append(stock_info)
                        
                except Exception as e:
                    print(f"Error processing {symbol}: {e}")

        return results

    def run_daily_analysis(self):
        """Run daily analysis with Excel output"""
        current_date = datetime.now().strftime('%Y%m%d')
        
        print("Loading stock lists...")
        nse_symbols = self.download_nse_stocks()
        bse_symbols = self.download_bse_stocks()

        print(f"\nAnalyzing {len(nse_symbols)} NSE and {len(bse_symbols)} BSE stocks")

        for exchange, symbols in [('NSE', nse_symbols), ('BSE', bse_symbols)]:
            batch_data=(symbols, exchange)
            results = self.process_stock_batch(batch_data)
            
            if results:
                # Create Excel writer with xlsxwriter engine
                excel_file = os.path.join(
                    self.results_dir,
                    f'{exchange.lower()}_analysis_{current_date}.xlsx'
                )
                
                # Convert results list to DataFrame
                results_df = pd.DataFrame(results)
                
                with pd.ExcelWriter(excel_file, engine='xlsxwriter') as writer:
                    # Write all stocks to first sheet
                    results_df.to_excel(writer, sheet_name='All Stocks', index=False)
                    
                    # Write high RSI stocks to second sheet
                    high_rsi = results_df[results_df['RSI'] >= 40].sort_values('RSI', ascending=False)
                    high_rsi.to_excel(writer, sheet_name='High RSI Stocks', index=False)
                    
                    # Get workbook and worksheet objects for formatting
                    workbook = writer.book
                    
                    # Add formats
                    header_format = workbook.add_format({
                        'bold': True,
                        'text_wrap': True,
                        'valign': 'top',
                        'bg_color': '#D9E1F2',
                        'border': 1
                    })
                    
                    # Format each worksheet
                    for worksheet in writer.sheets.values():
                        # Set column widths
                        worksheet.set_column('A:Z', 15)
                        # Apply header format to first row
                        for col_num, value in enumerate(results_df.columns.values):
                            worksheet.write(0, col_num, value, header_format)

                print(f"\n{exchange} analysis complete. Results saved to {excel_file}")
                
                # Print high RSI stocks
                print(f"\n{exchange} Stocks with RSI >= 40:")
                for _, row in high_rsi.iterrows():
                    print(colored(
                        f"{row['Symbol']} ({row['Company_Name']}): RSI = {row['RSI']:.2f}, "
                        f"Price = ₹{row['Latest_Price']:.2f}",
                        'green', attrs=['bold']
                    ))

        print(f"\nDaily analysis complete. Results saved in {self.results_dir}")

if __name__ == "__main__":
    analyzer = StockAnalyzer()
    analyzer.run_daily_analysis()

previous project(above code)

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from termcolor import colored
import requests
import io
import time
import os
import json
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
from functools import lru_cache

class StockAnalyzer:
    def __init__(self):
        self.base_dir = 'stock_analysis'
        self.stock_lists_dir = os.path.join(self.base_dir, 'stock_lists')
        self.results_dir = os.path.join(self.base_dir, 'daily_results')
        self.delisted_cache_file = os.path.join(self.stock_lists_dir, 'delisted_stocks.json')
        self.setup_directories()
        self.load_delisted_stocks()
        self.session = requests.Session()

    def setup_directories(self):
        """Create necessary directories if they don't exist"""
        os.makedirs(self.stock_lists_dir, exist_ok=True)
        os.makedirs(self.results_dir, exist_ok=True)
        
    def should_update_stock_list(self, exchange):
        """Check if stock list needs updating (weekly update)"""
        list_file = os.path.join(self.stock_lists_dir, f'{exchange}_stocks.json')
        if not os.path.exists(list_file):
            return True
        # Check if file is older than 7 days
        file_time = datetime.fromtimestamp(os.path.getmtime(list_file))
        return (datetime.now() - file_time).days >= 7
    
    def load_delisted_stocks(self):
        """Load previously identified delisted stocks"""
        try:
            with open(self.delisted_cache_file, 'r') as f:
                self.delisted_stocks = json.load(f)
        except FileNotFoundError:
            self.delisted_stocks = {'NSE': [], 'BSE': []}  # Changed from set to list
            self.save_delisted_stocks()

    def save_delisted_stocks(self):
        """Save delisted stocks to cache"""
        with open(self.delisted_cache_file, 'w') as f:
            json.dump({
                'NSE': self.delisted_stocks['NSE'],
                'BSE': self.delisted_stocks['BSE']
            }, f)
    
    def save_stock_list(self, symbols, exchange):
        """Save stock list to JSON file"""
        list_file = os.path.join(self.stock_lists_dir, f'{exchange}_stocks.json')
        data = {
            'symbols': symbols,
            'last_updated': datetime.now().strftime('%Y-%m-%d')
        }
        with open(list_file, 'w') as f:
            json.dump(data, f)

    def load_stock_list(self, exchange):
        """Load stock list from JSON file"""
        list_file = os.path.join(self.stock_lists_dir, f'{exchange}_stocks.json')
        with open(list_file, 'r') as f:
            data = json.load(f)
        return data['symbols']
    
    def download_nse_stocks(self):
        """Download NSE stock list if needed"""
        if self.should_update_stock_list('NSE'):
            try:
                url = "https://archives.nseindia.com/content/equities/EQUITY_L.csv"
                response = requests.get(url)
                df = pd.read_csv(io.StringIO(response.content.decode('utf-8')))
                symbols = df['SYMBOL'].tolist()
                self.save_stock_list(symbols, 'NSE')
                print("NSE stock list updated")
                return symbols
            except Exception as e:
                print(f"Error downloading NSE stocks: {e}")
                if os.path.exists(os.path.join(self.stock_lists_dir, 'NSE_stocks.json')):
                    return self.load_stock_list('NSE')
                return []
        return self.load_stock_list('NSE')

    def download_bse_stocks(self):
        """Download BSE stock list if needed"""
        if self.should_update_stock_list('BSE'):
            try:
                # Using the BSE list from static file or an appropriate source
                url = "https://www.bseindia.com/corporates/List_Scrips.aspx"
                response = requests.get(url)
                soup = BeautifulSoup(response.content, 'html.parser')
                table = soup.find('table', {'id': 'ContentPlaceHolder1_tblData'})
                symbols = []

                if table:
                    rows = table.find_all('tr')[1:]  # Skip header row
                    for row in rows:
                        cols = row.find_all('td')
                        if cols:
                            bse_code = cols[0].text.strip()
                            symbols.append(bse_code)

                if symbols:
                    self.save_stock_list(symbols, 'BSE')
                    print("BSE stock list updated")
                    return symbols
            except Exception as e:
                print(f"Error downloading BSE stocks: {e}")
                if os.path.exists(os.path.join(self.stock_lists_dir, 'BSE_stocks.json')):
                    return self.load_stock_list('BSE')
                return []
        return self.load_stock_list('BSE')

    @lru_cache(maxsize=1000)
    def calculate_rsi(self, prices_tuple, periods=14):
        """Calculate RSI with caching for performance"""
        prices = pd.Series(prices_tuple)
        delta = prices.diff()
        gain = delta.where(delta > 0, 0).rolling(window=periods).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=periods).mean()
        rs = gain / loss
        return float(100 - (100 / (1 + rs)).iloc[-1])


    def calculate_ema(self, data, length, source='Close', offset=0, smoothing_length=9):
        """
        Calculate Exponential Moving Average with specified parameters
        
        Args:
            data (pd.DataFrame): Historical price data
            length (int): Length of EMA period
            source (str): Source column for calculation
            offset (int): Offset for calculation
            smoothing_length (int): Length of initial SMA smoothing
            
        Returns:
            pd.Series: EMA values
        """
        # First calculate SMA as the smoothing line
        sma = data[source].rolling(window=smoothing_length).mean()
        
        # Calculate the multiplier
        multiplier = 2 / (length + 1)
        
        # Initialize EMA with SMA
        ema = pd.Series(index=data.index, dtype=float)
        ema.iloc[:length-1] = np.nan
        ema.iloc[length-1] = sma.iloc[length-1]
        
        # Calculate EMA
        for i in range(length, len(data)):
            ema.iloc[i] = (data[source].iloc[i] - ema.iloc[i-1]) * multiplier + ema.iloc[i-1]
        
        # Apply offset if specified
        if offset != 0:
            ema = ema.shift(offset)
            
        return ema

    def get_stock_data(self, symbol, exchange='NSE', days=60):
        """Fetch detailed stock data with additional information"""
        if symbol in self.delisted_stocks[exchange]:
            return None, None, None, None

        try:
            ticker_symbol = f"{symbol}.NS" if exchange == 'NSE' else f"{symbol}.BO"
            ticker = yf.Ticker(ticker_symbol)
            
            # Get stock info
            info = ticker.info
            
            end_date = datetime.now()
            start_date = end_date - timedelta(days=days)

            hist_data = ticker.history(
                start=start_date,
                end=end_date,
                interval="1d"
            )

            if hist_data.empty or len(hist_data) < 20:
                self.delisted_stocks[exchange].append(symbol)
                self.save_delisted_stocks()
                return None, None, None, None

            # Calculate both EMAs
            hist_data['EMA_9'] = self.calculate_ema(hist_data, length=9, source='Close', offset=0, smoothing_length=9)
            hist_data['EMA_21'] = self.calculate_ema(hist_data, length=21, source='Close', offset=0, smoothing_length=9)
            hist_data['EMA_50'] = self.calculate_ema(hist_data, length=50, source='Close', offset=0, smoothing_length=9)

            latest_price = hist_data['Close'].iloc[-1]
            prices_tuple = tuple(hist_data['Close'].values)
            latest_rsi = self.calculate_rsi(prices_tuple)

            return latest_price, hist_data, latest_rsi, info

        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")
            self.delisted_stocks[exchange].append(symbol)
            self.save_delisted_stocks()
            return None, None, None, None

    def process_stock_batch(self, batch_data):
        """Process a batch of stocks with enhanced information"""
        symbols, exchange = batch_data
        results = []
        
        with ThreadPoolExecutor(max_workers=10) as executor:
            futures = {executor.submit(self.get_stock_data, symbol, exchange): symbol 
                      for symbol in symbols}
            
            for future in futures:
                symbol = futures[future]
                try:
                    latest_price, hist_data, latest_rsi, info = future.result()
                    if all(v is not None for v in [latest_price, hist_data, latest_rsi, info]):
                        # Extract additional information safely using get() method
                        stock_info = {
                            'Symbol': symbol,
                            'Exchange': exchange,
                            'Company_Name': info.get('longName', 'N/A'),
                            'Latest_Price': latest_price,
                            'EMA_9': hist_data['EMA_9'].iloc[-1] if 'EMA_9' in hist_data else 'N/A',
                            'EMA_21': hist_data['EMA_21'].iloc[-1] if 'EMA_21' in hist_data else 'N/A',
                            'EMA_50': hist_data['EMA_50'].iloc[-1] if 'EMA_50' in hist_data else 'N/A',
                            'RSI': latest_rsi,
                            'Volume': hist_data['Volume'].iloc[-1] if 'Volume' in hist_data else 0,
                            'Market_Cap': info.get('marketCap', 'N/A'),
                            'PE_Ratio': info.get('trailingPE', 'N/A'),
                            'EPS': info.get('trailingEps', 'N/A'),
                            'Dividend_Yield': info.get('dividendYield', 'N/A'),
                            'Book_Value': info.get('bookValue', 'N/A'),
                            'Sector': info.get('sector', 'N/A'),
                            'Industry': info.get('industry', 'N/A'),
                            '52W_High': info.get('fiftyTwoWeekHigh', 'N/A'),
                            '52W_Low': info.get('fiftyTwoWeekLow', 'N/A'),
                            '50d_MA': info.get('fiftyDayAverage', 'N/A'),
                            '200d_MA': info.get('twoHundredDayAverage', 'N/A'),
                            'Beta': info.get('beta', 'N/A'),
                            'Previous_Close': info.get('previousClose', 'N/A'),
                            'Open': info.get('open', 'N/A'),
                            'Day_High': info.get('dayHigh', 'N/A'),
                            'Day_Low': info.get('dayLow', 'N/A'),
                            'Date': datetime.now().strftime('%Y-%m-%d')
                        }
                        
                        # Calculate additional metrics
                        if hist_data is not None:
                            stock_info.update({
                                'Volatility_30d': hist_data['Close'].pct_change().std() * np.sqrt(252),
                                'Returns_30d': (hist_data['Close'].iloc[-1] / hist_data['Close'].iloc[0] - 1) * 100,
                                'Average_Volume_30d': hist_data['Volume'].mean()
                            })
                        
                        results.append(stock_info)
                        
                except Exception as e:
                    print(f"Error processing {symbol}: {e}")

        return results

    def run_daily_analysis(self):
        """Run daily analysis with Excel output"""
        current_date = datetime.now().strftime('%Y%m%d')
        
        print("Loading stock lists...")
        nse_symbols = self.download_nse_stocks()
        bse_symbols = self.download_bse_stocks()

        print(f"\nAnalyzing {len(nse_symbols)} NSE and {len(bse_symbols)} BSE stocks")

        for exchange, symbols in [('NSE', nse_symbols), ('BSE', bse_symbols)]:
            batch_data=(symbols, exchange)
            results = self.process_stock_batch(batch_data)
            
            if results:
                # Create Excel writer with xlsxwriter engine
                excel_file = os.path.join(
                    self.results_dir,
                    f'{exchange.lower()}_analysis_{current_date}.xlsx'
                )
                
                # Convert results list to DataFrame
                results_df = pd.DataFrame(results)
                
                with pd.ExcelWriter(excel_file, engine='xlsxwriter') as writer:
                    # Write all stocks to first sheet
                    results_df.to_excel(writer, sheet_name='All Stocks', index=False)
                    
                    # Write high RSI stocks to second sheet
                    high_rsi = results_df[results_df['RSI'] >= 40].sort_values('RSI', ascending=False)
                    high_rsi.to_excel(writer, sheet_name='High RSI Stocks', index=False)
                    
                    # Get workbook and worksheet objects for formatting
                    workbook = writer.book
                    
                    # Add formats
                    header_format = workbook.add_format({
                        'bold': True,
                        'text_wrap': True,
                        'valign': 'top',
                        'bg_color': '#D9E1F2',
                        'border': 1
                    })
                    
                    # Format each worksheet
                    for worksheet in writer.sheets.values():
                        # Set column widths
                        worksheet.set_column('A:Z', 15)
                        # Apply header format to first row
                        for col_num, value in enumerate(results_df.columns.values):
                            worksheet.write(0, col_num, value, header_format)

                print(f"\n{exchange} analysis complete. Results saved to {excel_file}")
                
                # Print high RSI stocks
                print(f"\n{exchange} Stocks with RSI >= 40:")
                for _, row in high_rsi.iterrows():
                    print(colored(
                        f"{row['Symbol']} ({row['Company_Name']}): RSI = {row['RSI']:.2f}, "
                        f"Price = ₹{row['Latest_Price']:.2f}",
                        'green', attrs=['bold']
                    ))

        print(f"\nDaily analysis complete. Results saved in {self.results_dir}")


if __name__ == "__main__":
    analyzer = StockAnalyzer()
    analyzer.run_daily_analysis()

need to add code snippets in above part(need to do it tommorroe i.e.,today(10-01-2025))

In [3]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from termcolor import colored
import requests
import io
import time
import os
import json
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
from functools import lru_cache

class StockAnalyzer:
    def __init__(self):
        self.base_dir = 'stock_analysis'
        self.stock_lists_dir = os.path.join(self.base_dir, 'stock_lists')
        self.results_dir = os.path.join(self.base_dir, 'daily_results')
        self.delisted_cache_file = os.path.join(self.stock_lists_dir, 'delisted_stocks.json')
        self.setup_directories()
        self.load_delisted_stocks()
        self.session = requests.Session()

    def setup_directories(self):
        """Create necessary directories if they don't exist"""
        os.makedirs(self.stock_lists_dir, exist_ok=True)
        os.makedirs(self.results_dir, exist_ok=True)
        
    def should_update_stock_list(self, exchange):
        """Check if stock list needs updating (weekly update)"""
        list_file = os.path.join(self.stock_lists_dir, f'{exchange}_stocks.json')
        if not os.path.exists(list_file):
            return True
        # Check if file is older than 7 days
        file_time = datetime.fromtimestamp(os.path.getmtime(list_file))
        return (datetime.now() - file_time).days >= 7
    
    def load_delisted_stocks(self):
        """Load previously identified delisted stocks"""
        try:
            with open(self.delisted_cache_file, 'r') as f:
                self.delisted_stocks = json.load(f)
        except FileNotFoundError:
            self.delisted_stocks = {'NSE': [], 'BSE': []}  # Changed from set to list
            self.save_delisted_stocks()

    def save_delisted_stocks(self):
        """Save delisted stocks to cache"""
        with open(self.delisted_cache_file, 'w') as f:
            json.dump({
                'NSE': self.delisted_stocks['NSE'],
                'BSE': self.delisted_stocks['BSE']
            }, f)
    
    def save_stock_list(self, symbols, exchange):
        """Save stock list to JSON file"""
        list_file = os.path.join(self.stock_lists_dir, f'{exchange}_stocks.json')
        data = {
            'symbols': symbols,
            'last_updated': datetime.now().strftime('%Y-%m-%d')
        }
        with open(list_file, 'w') as f:
            json.dump(data, f)

    def load_stock_list(self, exchange):
        """Load stock list from JSON file"""
        list_file = os.path.join(self.stock_lists_dir, f'{exchange}_stocks.json')
        with open(list_file, 'r') as f:
            data = json.load(f)
        return data['symbols']
    
    def download_nse_stocks(self):
        """Download NSE stock list if needed"""
        if self.should_update_stock_list('NSE'):
            try:
                url = "https://archives.nseindia.com/content/equities/EQUITY_L.csv"
                response = requests.get(url)
                df = pd.read_csv(io.StringIO(response.content.decode('utf-8')))
                symbols = df['SYMBOL'].tolist()
                self.save_stock_list(symbols, 'NSE')
                print("NSE stock list updated")
                return symbols
            except Exception as e:
                print(f"Error downloading NSE stocks: {e}")
                if os.path.exists(os.path.join(self.stock_lists_dir, 'NSE_stocks.json')):
                    return self.load_stock_list('NSE')
                return []
        return self.load_stock_list('NSE')

    def download_bse_stocks(self):
        """Download BSE stock list if needed"""
        if self.should_update_stock_list('BSE'):
            try:
                # Using the BSE list from static file or an appropriate source
                url = "https://www.bseindia.com/corporates/List_Scrips.aspx"
                response = requests.get(url)
                soup = BeautifulSoup(response.content, 'html.parser')
                table = soup.find('table', {'id': 'ContentPlaceHolder1_tblData'})
                symbols = []

                if table:
                    rows = table.find_all('tr')[1:]  # Skip header row
                    for row in rows:
                        cols = row.find_all('td')
                        if cols:
                            bse_code = cols[0].text.strip()
                            symbols.append(bse_code)

                if symbols:
                    self.save_stock_list(symbols, 'BSE')
                    print("BSE stock list updated")
                    return symbols
            except Exception as e:
                print(f"Error downloading BSE stocks: {e}")
                if os.path.exists(os.path.join(self.stock_lists_dir, 'BSE_stocks.json')):
                    return self.load_stock_list('BSE')
                return []
        return self.load_stock_list('BSE')

    @lru_cache(maxsize=1000)
    def calculate_rsi(self, prices_tuple, periods=14):
        """Calculate RSI with caching for performance"""
        prices = pd.Series(prices_tuple)
        delta = prices.diff()
        gain = delta.where(delta > 0, 0).rolling(window=periods).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=periods).mean()
        rs = gain / loss
        return float(100 - (100 / (1 + rs)).iloc[-1])

    def calculate_ema(self, data, length, source='Close', offset=0, smoothing_length=9):
        """
        Calculate Exponential Moving Average with specified parameters
        
        Args:
            data (pd.DataFrame): Historical price data
            length (int): Length of EMA period
            source (str): Source column for calculation
            offset (int): Offset for calculation
            smoothing_length (int): Length of initial SMA smoothing
            
        Returns:
            pd.Series: EMA values
        """
        # First calculate SMA as the smoothing line
        sma = data[source].rolling(window=smoothing_length).mean()
        
        # Calculate the multiplier
        multiplier = 2 / (length + 1)
        
        # Initialize EMA with SMA
        ema = pd.Series(index=data.index, dtype=float)
        ema.iloc[:length-1] = np.nan
        ema.iloc[length-1] = sma.iloc[length-1]
        
        # Calculate EMA
        for i in range(length, len(data)):
            ema.iloc[i] = (data[source].iloc[i] - ema.iloc[i-1]) * multiplier + ema.iloc[i-1]
        
        # Apply offset if specified
        if offset != 0:
            ema = ema.shift(offset)
            
        return ema

    def get_stock_data(self, symbol, exchange='NSE', days=100):
        """Fetch detailed stock data with additional information"""
        if symbol in self.delisted_stocks[exchange]:
            return None, None, None, None

        try:
            ticker_symbol = f"{symbol}.NS" if exchange == 'NSE' else f"{symbol}.BO"
            ticker = yf.Ticker(ticker_symbol)
            
            # Get stock info
            info = ticker.info
            
            end_date = datetime.now()
            start_date = end_date - timedelta(days=days)

            hist_data = ticker.history(
                start=start_date,
                end=end_date,
                interval="1d"
            )

            if hist_data.empty or len(hist_data) < 20:
                self.delisted_stocks[exchange].append(symbol)
                self.save_delisted_stocks()
                return None, None, None, None

            # Calculate all three EMAs
            hist_data['EMA_9'] = self.calculate_ema(hist_data, length=9, source='Close', offset=0, smoothing_length=9)
            hist_data['EMA_21'] = self.calculate_ema(hist_data, length=21, source='Close', offset=0, smoothing_length=9)
            hist_data['EMA_50'] = self.calculate_ema(hist_data, length=50, source='Close', offset=0, smoothing_length=9)

            latest_price = hist_data['Close'].iloc[-1]
            prices_tuple = tuple(hist_data['Close'].values)
            latest_rsi = self.calculate_rsi(prices_tuple)

            return latest_price, hist_data, latest_rsi, info

        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")
            self.delisted_stocks[exchange].append(symbol)
            self.save_delisted_stocks()
            return None, None, None, None

    def process_stock_batch(self, batch_data):
        """Process a batch of stocks with enhanced information"""
        symbols, exchange = batch_data
        results = []
        
        with ThreadPoolExecutor(max_workers=10) as executor:
            futures = {executor.submit(self.get_stock_data, symbol, exchange): symbol 
                      for symbol in symbols}
            
            for future in futures:
                symbol = futures[future]
                try:
                    latest_price, hist_data, latest_rsi, info = future.result()
                    if all(v is not None for v in [latest_price, hist_data, latest_rsi, info]):
                        # Extract additional information safely using get() method
                        stock_info = {
                            'Symbol': symbol,
                            'Exchange': exchange,
                            'Company_Name': info.get('longName', 'N/A'),
                            'Latest_Price': latest_price,
                            'EMA_9': hist_data['EMA_9'].iloc[-1] if 'EMA_9' in hist_data else 'N/A',
                            'EMA_21': hist_data['EMA_21'].iloc[-1] if 'EMA_21' in hist_data else 'N/A',
                            'EMA_50': hist_data['EMA_50'].iloc[-1] if 'EMA_50' in hist_data else 'N/A',
                            'RSI': latest_rsi,
                            'Volume': hist_data['Volume'].iloc[-1] if 'Volume' in hist_data else 0,
                            'Market_Cap': info.get('marketCap', 'N/A'),
                            'PE_Ratio': info.get('trailingPE', 'N/A'),
                            'EPS': info.get('trailingEps', 'N/A'),
                            'Dividend_Yield': info.get('dividendYield', 'N/A'),
                            'Book_Value': info.get('bookValue', 'N/A'),
                            'Sector': info.get('sector', 'N/A'),
                            'Industry': info.get('industry', 'N/A'),
                            '52W_High': info.get('fiftyTwoWeekHigh', 'N/A'),
                            '52W_Low': info.get('fiftyTwoWeekLow', 'N/A'),
                            '50d_MA': info.get('fiftyDayAverage', 'N/A'),
                            '200d_MA': info.get('twoHundredDayAverage', 'N/A'),
                            'Beta': info.get('beta', 'N/A'),
                            'Previous_Close': info.get('previousClose', 'N/A'),
                            'Open': info.get('open', 'N/A'),
                            'Day_High': info.get('dayHigh', 'N/A'),
                            'Day_Low': info.get('dayLow', 'N/A'),
                            'Date': datetime.now().strftime('%Y-%m-%d')
                        }
                        
                        # Calculate additional metrics
                        if hist_data is not None:
                            stock_info.update({
                                'Volatility_30d': hist_data['Close'].pct_change().std() * np.sqrt(252),
                                'Returns_30d': (hist_data['Close'].iloc[-1] / hist_data['Close'].iloc[0] - 1) * 100,
                                'Average_Volume_30d': hist_data['Volume'].mean()
                            })
                        
                        results.append(stock_info)
                        
                except Exception as e:
                    print(f"Error processing {symbol}: {e}")

        return results

    def run_daily_analysis(self):
        """Run daily analysis with Excel output"""
        current_date = datetime.now().strftime('%Y%m%d')
        
        print("Loading stock lists...")
        nse_symbols = self.download_nse_stocks()
        bse_symbols = self.download_bse_stocks()

        print(f"\nAnalyzing {len(nse_symbols)} NSE and {len(bse_symbols)} BSE stocks")

        for exchange, symbols in [('NSE', nse_symbols), ('BSE', bse_symbols)]:
            batch_data=(symbols, exchange)
            results = self.process_stock_batch(batch_data)
            
            if results:
                # Create Excel writer with xlsxwriter engine
                excel_file = os.path.join(
                    self.results_dir,
                    f'{exchange.lower()}_analysis_{current_date}.xlsx'
                )
                
                # Convert results list to DataFrame
                results_df = pd.DataFrame(results)
                
                with pd.ExcelWriter(excel_file, engine='xlsxwriter') as writer:
                    # Write all stocks to first sheet
                    results_df.to_excel(writer, sheet_name='All Stocks', index=False)
                    
                    # Write high RSI stocks to second sheet
                    high_rsi = results_df[results_df['RSI'] >= 40].sort_values('RSI', ascending=False)
                    high_rsi.to_excel(writer, sheet_name='High RSI Stocks', index=False)
                    
                    # Get workbook and worksheet objects for formatting
                    workbook = writer.book
                    
                    # Add formats
                    header_format = workbook.add_format({
                        'bold': True,
                        'text_wrap': True,
                        'valign': 'top',
                        'bg_color': '#D9E1F2',
                        'border': 1
                    })
                    
                    # Format each worksheet
                    for worksheet in writer.sheets.values():
                        # Set column widths
                        worksheet.set_column('A:Z', 15)
                        # Apply header format to first row
                        for col_num, value in enumerate(results_df.columns.values):
                            worksheet.write(0, col_num, value, header_format)

                print(f"\n{exchange} analysis complete. Results saved to {excel_file}")
                
                # Print high RSI stocks
                print(f"\n{exchange} Stocks with RSI >= 40:")
                for _, row in high_rsi.iterrows():
                    print(colored(
                        f"{row['Symbol']} ({row['Company_Name']}): RSI = {row['RSI']:.2f}, "
                        f"Price = ₹{row['Latest_Price']:.2f}",
                        'green', attrs=['bold']
                    ))

        print(f"\nDaily analysis complete. Results saved in {self.results_dir}")

if __name__ == "__main__":
    analyzer = StockAnalyzer()
    analyzer.run_daily_analysis()

Loading stock lists...

Analyzing 2062 NSE and 3026 BSE stocks

Daily analysis complete. Results saved in stock_analysis\daily_results


In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from termcolor import colored
import requests
import io
import time
import os
import json
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
from functools import lru_cache

class StockAnalyzer:
    def __init__(self):
        self.base_dir = 'stock_analysis'
        self.stock_lists_dir = os.path.join(self.base_dir, 'stock_lists')
        self.results_dir = os.path.join(self.base_dir, 'daily_results')
        self.delisted_cache_file = os.path.join(self.stock_lists_dir, 'delisted_stocks.json')
        self.setup_directories()
        self.load_delisted_stocks()
        self.session = requests.Session()

    def setup_directories(self):
        """Create necessary directories if they don't exist"""
        os.makedirs(self.stock_lists_dir, exist_ok=True)
        os.makedirs(self.results_dir, exist_ok=True)
        
    def should_update_stock_list(self, exchange):
        """Check if stock list needs updating (weekly update)"""
        list_file = os.path.join(self.stock_lists_dir, f'{exchange}_stocks.json')
        if not os.path.exists(list_file):
            return True
        # Check if file is older than 7 days
        file_time = datetime.fromtimestamp(os.path.getmtime(list_file))
        return (datetime.now() - file_time).days >= 7
    
    def load_delisted_stocks(self):
        """Load previously identified delisted stocks"""
        try:
            with open(self.delisted_cache_file, 'r') as f:
                self.delisted_stocks = json.load(f)
        except FileNotFoundError:
            self.delisted_stocks = {'NSE': [], 'BSE': []}  # Changed from set to list
            self.save_delisted_stocks()

    def save_delisted_stocks(self):
        """Save delisted stocks to cache"""
        with open(self.delisted_cache_file, 'w') as f:
            json.dump({
                'NSE': self.delisted_stocks['NSE'],
                'BSE': self.delisted_stocks['BSE']
            }, f)
    
    def save_stock_list(self, symbols, exchange):
        """Save stock list to JSON file"""
        list_file = os.path.join(self.stock_lists_dir, f'{exchange}_stocks.json')
        data = {
            'symbols': symbols,
            'last_updated': datetime.now().strftime('%Y-%m-%d')
        }
        with open(list_file, 'w') as f:
            json.dump(data, f)

    def load_stock_list(self, exchange):
        """Load stock list from JSON file"""
        list_file = os.path.join(self.stock_lists_dir, f'{exchange}_stocks.json')
        with open(list_file, 'r') as f:
            data = json.load(f)
        return data['symbols']
    
    def download_nse_stocks(self):
        """Download NSE stock list if needed"""
        if self.should_update_stock_list('NSE'):
            try:
                url = "https://archives.nseindia.com/content/equities/EQUITY_L.csv"
                response = requests.get(url)
                df = pd.read_csv(io.StringIO(response.content.decode('utf-8')))
                symbols = df['SYMBOL'].tolist()
                self.save_stock_list(symbols, 'NSE')
                print("NSE stock list updated")
                return symbols
            except Exception as e:
                print(f"Error downloading NSE stocks: {e}")
                if os.path.exists(os.path.join(self.stock_lists_dir, 'NSE_stocks.json')):
                    return self.load_stock_list('NSE')
                return []
        return self.load_stock_list('NSE')

    def download_bse_stocks(self):
        """Download BSE stock list if needed"""
        if self.should_update_stock_list('BSE'):
            try:
                # Using the BSE list from static file or an appropriate source
                url = "https://www.bseindia.com/corporates/List_Scrips.aspx"
                response = requests.get(url)
                soup = BeautifulSoup(response.content, 'html.parser')
                table = soup.find('table', {'id': 'ContentPlaceHolder1_tblData'})
                symbols = []

                if table:
                    rows = table.find_all('tr')[1:]  # Skip header row
                    for row in rows:
                        cols = row.find_all('td')
                        if cols:
                            bse_code = cols[0].text.strip()
                            symbols.append(bse_code)

                if symbols:
                    self.save_stock_list(symbols, 'BSE')
                    print("BSE stock list updated")
                    return symbols
            except Exception as e:
                print(f"Error downloading BSE stocks: {e}")
                if os.path.exists(os.path.join(self.stock_lists_dir, 'BSE_stocks.json')):
                    return self.load_stock_list('BSE')
                return []
        return self.load_stock_list('BSE')

    @lru_cache(maxsize=1000)
    def calculate_rsi(self, prices_tuple, periods=14):
        """Calculate RSI with caching for performance"""
        prices = pd.Series(prices_tuple)
        delta = prices.diff()
        gain = delta.where(delta > 0, 0).rolling(window=periods).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=periods).mean()
        rs = gain / loss
        return float(100 - (100 / (1 + rs)).iloc[-1])

    def calculate_ema(self, data, length, source='Close', offset=0, smoothing_length=9):
        """
        Calculate Exponential Moving Average with specified parameters
        
        Args:
            data (pd.DataFrame): Historical price data
            length (int): Length of EMA period
            source (str): Source column for calculation
            offset (int): Offset for calculation
            smoothing_length (int): Length of initial SMA smoothing
            
        Returns:
            pd.Series: EMA values
        """
        # First calculate SMA as the smoothing line
        sma = data[source].rolling(window=smoothing_length).mean()
        
        # Calculate the multiplier
        multiplier = 2 / (length + 1)
        
        # Initialize EMA with SMA
        ema = pd.Series(index=data.index, dtype=float)
        ema.iloc[:length-1] = np.nan
        ema.iloc[length-1] = sma.iloc[length-1]
        
        # Calculate EMA
        for i in range(length, len(data)):
            ema.iloc[i] = (data[source].iloc[i] - ema.iloc[i-1]) * multiplier + ema.iloc[i-1]
        
        # Apply offset if specified
        if offset != 0:
            ema = ema.shift(offset)
            
        return ema

    def get_stock_data(self, symbol, exchange='NSE', days=200):
        """Fetch detailed stock data with additional information"""
        if symbol in self.delisted_stocks[exchange]:
            return None, None, None, None

        try:
            time.sleep(1)  # Add a delay of 1 second between requests
            ticker_symbol = f"{symbol}.NS" if exchange == 'NSE' else f"{symbol}.BO"
            ticker = yf.Ticker(ticker_symbol)
            
            # Get stock info
            info = ticker.info
            
            end_date = datetime.now()
            start_date = end_date - timedelta(days=days)

            hist_data = ticker.history(
                start=start_date,
                end=end_date,
                interval="1d"
            )

            if hist_data.empty or len(hist_data) < 20:
                self.delisted_stocks[exchange].append(symbol)
                self.save_delisted_stocks()
                return None, None, None, None

            # Calculate all three EMAs
            hist_data['EMA_9'] = self.calculate_ema(hist_data, length=9, source='Close', offset=0, smoothing_length=9)
            hist_data['EMA_21'] = self.calculate_ema(hist_data, length=21, source='Close', offset=0, smoothing_length=9)
            hist_data['EMA_50'] = self.calculate_ema(hist_data, length=50, source='Close', offset=0, smoothing_length=9)

            latest_price = hist_data['Close'].iloc[-1]
            prices_tuple = tuple(hist_data['Close'].values)
            latest_rsi = self.calculate_rsi(prices_tuple)

            return latest_price, hist_data, latest_rsi, info

        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")
            self.delisted_stocks[exchange].append(symbol)
            self.save_delisted_stocks()
            return None, None, None, None

    def process_stock_batch(self, batch_data):
        """Process a batch of stocks with enhanced information"""
        symbols, exchange = batch_data
        results = []
        
        with ThreadPoolExecutor(max_workers=8) as executor:
            futures = {executor.submit(self.get_stock_data, symbol, exchange): symbol 
                      for symbol in symbols}
            
            for future in futures:
                symbol = futures[future]
                try:
                    latest_price, hist_data, latest_rsi, info = future.result()
                    if all(v is not None for v in [latest_price, hist_data, latest_rsi, info]):
                        # Extract additional information safely using get() method
                        stock_info = {
                            'Symbol': symbol,
                            'Exchange': exchange,
                            'Company_Name': info.get('longName', 'N/A'),
                            'Latest_Price': latest_price,
                            'EMA_9': hist_data['EMA_9'].iloc[-1] if 'EMA_9' in hist_data else 'N/A',
                            'EMA_21': hist_data['EMA_21'].iloc[-1] if 'EMA_21' in hist_data else 'N/A',
                            'EMA_50': hist_data['EMA_50'].iloc[-1] if 'EMA_50' in hist_data else 'N/A',
                            'RSI': latest_rsi,
                            'Volume': hist_data['Volume'].iloc[-1] if 'Volume' in hist_data else 0,
                            'Market_Cap': info.get('marketCap', 'N/A'),
                            'PE_Ratio': info.get('trailingPE', 'N/A'),
                            'EPS': info.get('trailingEps', 'N/A'),
                            'Dividend_Yield': info.get('dividendYield', 'N/A'),
                            'Book_Value': info.get('bookValue', 'N/A'),
                            'Sector': info.get('sector', 'N/A'),
                            'Industry': info.get('industry', 'N/A'),
                            '52W_High': info.get('fiftyTwoWeekHigh', 'N/A'),
                            '52W_Low': info.get('fiftyTwoWeekLow', 'N/A'),
                            '50d_MA': info.get('fiftyDayAverage', 'N/A'),
                            '200d_MA': info.get('twoHundredDayAverage', 'N/A'),
                            'Beta': info.get('beta', 'N/A'),
                            'Previous_Close': info.get('previousClose', 'N/A'),
                            'Open': info.get('open', 'N/A'),
                            'Day_High': info.get('dayHigh', 'N/A'),
                            'Day_Low': info.get('dayLow', 'N/A'),
                            'Date': datetime.now().strftime('%Y-%m-%d')
                        }
                        
                        # Calculate additional metrics
                        if hist_data is not None:
                            stock_info.update({
                                'Volatility_30d': hist_data['Close'].pct_change().std() * np.sqrt(252),
                                'Returns_30d': (hist_data['Close'].iloc[-1] / hist_data['Close'].iloc[0] - 1) * 100,
                                'Average_Volume_30d': hist_data['Volume'].mean()
                            })
                        
                        results.append(stock_info)
                        
                except Exception as e:
                    print(f"Error processing {symbol}: {e}")

        return results

    def run_daily_analysis(self):
        """Run daily analysis with Excel output"""
        current_date = datetime.now().strftime('%Y%m%d')
        
        print("Loading stock lists...")
        nse_symbols = self.download_nse_stocks()
        bse_symbols = self.download_bse_stocks()

        print(f"\nAnalyzing {len(nse_symbols)} NSE and {len(bse_symbols)} BSE stocks")

        for exchange, symbols in [('NSE', nse_symbols), ('BSE', bse_symbols)]:
            batch_data=(symbols, exchange)
            results = self.process_stock_batch(batch_data)
            
            if results:
                # Create Excel writer with xlsxwriter engine
                excel_file = os.path.join(
                    self.results_dir,
                    f'{exchange.lower()}_analysis_{current_date}.xlsx'
                )
                
                # Convert results list to DataFrame
                results_df = pd.DataFrame(results)
                
                with pd.ExcelWriter(excel_file, engine='xlsxwriter') as writer:
                    # Write all stocks to first sheet
                    results_df.to_excel(writer, sheet_name='All Stocks', index=False)
                    
                    # Write high RSI stocks to second sheet
                    high_rsi = results_df[results_df['RSI'] >= 40].sort_values('RSI', ascending=False)
                    high_rsi.to_excel(writer, sheet_name='High RSI Stocks', index=False)
                    
                    # Get workbook and worksheet objects for formatting
                    workbook = writer.book
                    
                    # Add formats
                    header_format = workbook.add_format({
                        'bold': True,
                        'text_wrap': True,
                        'valign': 'top',
                        'bg_color': '#D9E1F2',
                        'border': 1
                    })
                    
                    # Format each worksheet
                    for worksheet in writer.sheets.values():
                        # Set column widths
                        worksheet.set_column('A:Z', 15)
                        # Apply header format to first row
                        for col_num, value in enumerate(results_df.columns.values):
                            worksheet.write(0, col_num, value, header_format)

                print(f"\n{exchange} analysis complete. Results saved to {excel_file}")
                
                # Print high RSI stocks
                print(f"\n{exchange} Stocks with RSI >= 40:")
                for _, row in high_rsi.iterrows():
                    print(colored(
                        f"{row['Symbol']} ({row['Company_Name']}): RSI = {row['RSI']:.2f}, "
                        f"Price = ₹{row['Latest_Price']:.2f}",
                        'green', attrs=['bold']
                    ))

        print(f"\nDaily analysis complete. Results saved in {self.results_dir}")

if __name__ == "__main__":
    analyzer = StockAnalyzer()
    analyzer.run_daily_analysis()

Loading stock lists...

Analyzing 2062 NSE and 3026 BSE stocks


$ICICIBANK.NS: possibly delisted; no price data found  (1d 2024-06-24 13:22:09.188021 -> 2025-01-10 13:22:09.188021)


Error fetching data for MASTERTR: single positional indexer is out-of-bounds


$MRF.NS: possibly delisted; no price data found  (1d 2024-06-24 13:23:53.392810 -> 2025-01-10 13:23:53.392810)


Error fetching data for RACLGEAR: single positional indexer is out-of-bounds
Error fetching data for ROSSTECH: single positional indexer is out-of-bounds
Error fetching data for SAGILITY: single positional indexer is out-of-bounds
Error fetching data for SURAKSHA: single positional indexer is out-of-bounds
Error fetching data for SWIGGY: single positional indexer is out-of-bounds

NSE analysis complete. Results saved to stock_analysis\daily_results\nse_analysis_20250110.xlsx

NSE Stocks with RSI >= 40:
SVLL (Shree Vasu Logistics Limited): RSI = 100.00, Price = ₹394.65
VISASTEEL (VISA Steel Limited): RSI = 100.00, Price = ₹45.48
GVT&D (GE Vernova T&D India Ltd): RSI = 100.00, Price = ₹1956.30
BLUECOAST (Blue Coast Hotels Limited): RSI = 100.00, Price = ₹22.27
BGRENERGY (BGR Energy Systems Limited): RSI = 100.00, Price = ₹119.00
AARVEEDEN (Aarvee Denims and Exports Ltd.): RSI = 100.00, Price = ₹131.58
ATLASCYCLE (Atlas Cycles (Haryana) Limited): RSI = 100.00, Price = ₹112.81
FSC (Future 

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from termcolor import colored
import requests
import io
import time
import os
import json
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
from functools import lru_cache

class StockAnalyzer:
    def __init__(self):
        self.base_dir = 'stock_analysis'
        self.stock_lists_dir = os.path.join(self.base_dir, 'stock_lists')
        self.results_dir = os.path.join(self.base_dir, 'daily_results')
        self.delisted_cache_file = os.path.join(self.stock_lists_dir, 'delisted_stocks.json')
        self.setup_directories()
        self.load_delisted_stocks()
        self.session = requests.Session()

    # [Previous setup methods remain unchanged...]

    def calculate_ema(self, data, length, source='Close', offset=0, smoothing_length=9):
        """
        Calculate Exponential Moving Average with specified parameters and error handling
        """
        try:
            if len(data) < max(length, smoothing_length):
                return pd.Series(index=data.index, dtype=float)
            
            # First calculate SMA as the smoothing line
            sma = data[source].rolling(window=smoothing_length, min_periods=1).mean()
            
            # Calculate the multiplier
            multiplier = 2 / (length + 1)
            
            # Initialize EMA with SMA
            ema = pd.Series(index=data.index, dtype=float)
            ema.iloc[:length-1] = np.nan
            
            # Make sure we have enough data for the initial value
            if len(sma.dropna()) > length-1:
                ema.iloc[length-1] = sma.iloc[length-1]
                
                # Calculate EMA
                for i in range(length, len(data)):
                    ema.iloc[i] = (data[source].iloc[i] - ema.iloc[i-1]) * multiplier + ema.iloc[i-1]
                
                # Apply offset if specified
                if offset != 0:
                    ema = ema.shift(offset)
            
            return ema
            
        except Exception as e:
            print(f"Error calculating EMA: {e}")
            return pd.Series(index=data.index, dtype=float)

    def get_stock_data(self, symbol, exchange='NSE', days=100):  # Increased days to ensure enough data
        """Fetch detailed stock data with additional information"""
        if symbol in self.delisted_stocks[exchange]:
            return None, None, None, None

        try:
            ticker_symbol = f"{symbol}.NS" if exchange == 'NSE' else f"{symbol}.BO"
            ticker = yf.Ticker(ticker_symbol)
            
            # Get stock info
            info = ticker.info
            
            end_date = datetime.now()
            start_date = end_date - timedelta(days=days)

            hist_data = ticker.history(
                start=start_date,
                end=end_date,
                interval="1d"
            )

            if hist_data.empty or len(hist_data) < 20:
                self.delisted_stocks[exchange].append(symbol)
                self.save_delisted_stocks()
                return None, None, None, None

            # Calculate EMAs with error handling
            try:
                hist_data['EMA_9'] = self.calculate_ema(hist_data, length=9, source='Close', offset=0, smoothing_length=9)
                hist_data['EMA_21'] = self.calculate_ema(hist_data, length=21, source='Close', offset=0, smoothing_length=9)
                hist_data['EMA_50'] = self.calculate_ema(hist_data, length=50, source='Close', offset=0, smoothing_length=9)
            except Exception as e:
                print(f"Error calculating EMAs for {symbol}: {e}")
                hist_data['EMA_9'] = np.nan
                hist_data['EMA_21'] = np.nan
                hist_data['EMA_50'] = np.nan

            latest_price = hist_data['Close'].iloc[-1] if not hist_data.empty else None
            prices_tuple = tuple(hist_data['Close'].values) if not hist_data.empty else ()
            latest_rsi = self.calculate_rsi(prices_tuple) if prices_tuple else None

            return latest_price, hist_data, latest_rsi, info

        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")
            return None, None, None, None

    def process_stock_batch(self, batch_data):
        """Process a batch of stocks with enhanced information"""
        symbols, exchange = batch_data
        results = []
        
        with ThreadPoolExecutor(max_workers=10) as executor:
            futures = {executor.submit(self.get_stock_data, symbol, exchange): symbol 
                      for symbol in symbols}
            
            for future in futures:
                symbol = futures[future]
                try:
                    latest_price, hist_data, latest_rsi, info = future.result()
                    if all(v is not None for v in [latest_price, hist_data, latest_rsi, info]):
                        # Get the latest available EMA values safely
                        ema_9 = hist_data['EMA_9'].iloc[-1] if not hist_data['EMA_9'].empty and not pd.isna(hist_data['EMA_9'].iloc[-1]) else 'N/A'
                        ema_21 = hist_data['EMA_21'].iloc[-1] if not hist_data['EMA_21'].empty and not pd.isna(hist_data['EMA_21'].iloc[-1]) else 'N/A'
                        ema_50 = hist_data['EMA_50'].iloc[-1] if not hist_data['EMA_50'].empty and not pd.isna(hist_data['EMA_50'].iloc[-1]) else 'N/A'
                        
                        stock_info = {
                            'Symbol': symbol,
                            'Exchange': exchange,
                            'Company_Name': info.get('longName', 'N/A'),
                            'Latest_Price': latest_price,
                            'EMA_9': ema_9,
                            'EMA_21': ema_21,
                            'EMA_50': ema_50,
                            'RSI': latest_rsi,
                            # [Rest of the fields remain the same...]
                        }
                        
                        # Calculate additional metrics
                        if hist_data is not None and not hist_data.empty:
                            stock_info.update({
                                'Volatility_30d': hist_data['Close'].pct_change().std() * np.sqrt(252),
                                'Returns_30d': (hist_data['Close'].iloc[-1] / hist_data['Close'].iloc[0] - 1) * 100,
                                'Average_Volume_30d': hist_data['Volume'].mean()
                            })
                        
                        results.append(stock_info)
                        
                except Exception as e:
                    print(f"Error processing {symbol}: {e}")

        return results

    # [Rest of the class methods remain unchanged...]

if __name__ == "__main__":
    analyzer = StockAnalyzer()
    analyzer.run_daily_analysis()